In [11]:
import pandas as pd
import os
import matplotlib.pyplot as plt
import func_stat

In [13]:
dirCat=r'D:\Sources\COSMOS_photoZ\COSMOS_v3.0\catalogs\forPaper\v05'
dirConfigs=r'D:\Sources\COSMOS_photoZ\COSMOS_v3.0\configsArticle\Vis_UV_IR'
dirExp=r'D:\Sources\COSMOS_photoZ\COSMOS_v3.0\experimentsArticle\Vis_UV_IR'
expNum=1
fileNameCat=r'05_COSMOS_SpectrZ_Zlim_1p2_AllBands_colors.csv'
idcolumn='Seq'
zspeccolumn='z_spec'
cat=pd.read_csv(os.path.join(dirCat,fileNameCat))

In [ ]:
for dExp in os.listdir(dirExp):
    os.path.join(dirExp,dExp)